In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
def extract_layer_output(model, layer_name, data):
    int_layer_op = model.get_layer(name = layer_name).output
    int_layer_model = Model(inputs = model.input, outputs = int_layer_op)
    int_layer_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return int_layer_model.predict(data)

In [5]:
def generate_df(arr, label):
    df = pd.DataFrame(arr)
    df['label'] = label
    return df

In [6]:
base_path = 'D:\\fake-vs-real\\real_vs_fake\\real-vs-fake\\'
image_gen = ImageDataGenerator(rescale=1./255.)

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    shuffle=False,
    class_mode='binary'
)

test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=1,
    shuffle = False,
    class_mode='binary'
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [7]:
y_test = test_flow.classes
y_test

array([0, 0, 0, ..., 1, 1, 1])

In [8]:
custom_model = load_model('deep_fake_custom_CNN_model.h5')

In [10]:
int_output_custom = extract_layer_output(custom_model, 'global_average_pooling2d_1', train_flow)
int_output_custom.shape

(100000, 1024)

In [11]:
int_output_custom = generate_df(int_output_custom, train_flow.classes)
int_output_custom.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,label
0,-0.102400,0.101428,0.235823,-0.215722,-0.044314,-0.043952,0.459600,-0.260262,-0.372854,0.151030,...,0.095684,0.208616,0.759955,0.633904,-0.610727,-0.174793,-0.246132,0.655625,-0.235347,0
1,0.026694,-0.498493,0.239279,-0.154200,-0.631037,0.502552,-0.144780,-0.090217,-0.004471,-0.419060,...,-0.565320,-0.437589,-0.259084,-0.017838,-0.152009,-0.474074,0.631287,-0.277794,-0.515764,0
2,-0.206004,-0.353236,-0.489914,-0.470469,0.137303,0.268092,0.357659,0.876869,0.223182,-0.105941,...,-0.459861,0.037206,-0.039159,0.431013,-0.061638,-0.395189,-0.000422,0.032739,0.060721,0
3,0.183939,-0.538174,-0.230662,-0.514412,0.211349,-0.305276,-0.117885,0.039673,-0.361994,-0.310106,...,-0.198161,-0.094567,-0.370660,0.029272,-0.691627,-0.696342,0.133271,0.121235,-0.119125,0
4,-0.515688,-0.361801,0.049795,0.130015,-0.350609,0.470957,0.004543,0.278675,-0.018741,0.308987,...,-0.072855,-0.049746,-0.635361,-0.214580,-0.275176,-0.740658,0.809420,0.300907,-0.012254,0


In [12]:
int_output_custom.to_csv('custom_model.csv', index = False)

In [13]:
y_pred_custom = custom_model.predict(test_flow)

In [14]:
print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred_custom))
print("Average Precision Score:", metrics.average_precision_score(y_test, y_pred_custom))
print()
print(metrics.classification_report(y_test, y_pred_custom > 0.5))

ROC AUC Score: 0.9885498899999999
Average Precision Score: 0.9874157360683484

              precision    recall  f1-score   support

           0       0.86      0.98      0.92     10000
           1       0.98      0.84      0.90     10000

    accuracy                           0.91     20000
   macro avg       0.92      0.91      0.91     20000
weighted avg       0.92      0.91      0.91     20000

